In [1]:
import numpy as np
import tensorflow as tf
import gym
import maxapproxi

np.random.seed(2)
tf.set_random_seed(2)  # reproducible

# Superparameters
OUTPUT_GRAPH = False
MAX_EPISODE = 3000
DISPLAY_REWARD_THRESHOLD = 200  # renders environment if total episode reward is greater then this threshold
MAX_EP_STEPS = 1000   # maximum time step in one episode
RENDER = False  # rendering wastes time
GAMMA = 0.9     # reward discount in TD error
ALPHA = 1.0 
LR_A = 0.001    # learning rate for actor
LR_C = 0.01     # learning rate for critic
env = gym.make('CartPole-v0')
env.seed(1)  # reproducible
env = env.unwrapped

N_F = env.observation_space.shape[0]
N_A = env.action_space.n

WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.


In [2]:
class ExpReplay(object):
    def __init__(self, memory_size, state_dim, act_dim, batch_size):
        self.memory_size = memory_size
        self.batch_size = batch_size
        self.state_dim = state_dim
        self.act_dim = act_dim
        self.states = np.empty((self.memory_size, self.state_dim), dtype=np.float32)
        self.actions = np.empty((self.memory_size, 1), dtype=np.float32)
        self.rewards = np.empty(self.memory_size, dtype=np.float32)
        self.next_states = np.empty((self.memory_size, self.state_dim), dtype=np.float32)
        self.count = 0
        self.current = 0
    def fifo(self, state, action, reward, next_state):
        self.states[self.current] = state
        self.actions[self.current] = action
        self.rewards[self.current] = reward
        self.next_states[self.current] = next_state
        self.current = (self.current + 1) % self.memory_size
        self.count = self.count + 1
    def add_trajectory(self, states, actions, rewards, next_states):
        num = len(observes)
        for i in range(0, num):
            self.fifo(states[i], actions[i], rewards[i], next_states[i])
    def sampling(self):
        indexes = np.random.randint(min(self.count, self.memory_size), size=self.batch_size)
        states = self.states[indexes]
        actions = self.actions[indexes]
        rewards = self.rewards[indexes]
        next_states = self.next_states[indexes]
        states = states.reshape((-1, self.state_dim))
        actions = actions.reshape((-1, 1))
        rewards = rewards.reshape((-1, 1))
        next_states = next_states.reshape((-1, self.state_dim))
        return states, actions, rewards, next_states

In [3]:
class Actor(object):
    def __init__(self, sess, n_features, n_actions, lr=0.001):
        self.sess = sess

        self.s = tf.placeholder(tf.float32, (None, n_features), "state")
        self.a = tf.placeholder(tf.float32, (None, n_actions), "act")
        self.td_error = tf.placeholder(tf.float32, (None, 1), "td_error")  # TD_error]
        self.eps = 1
        self.eps_decay_rate = 0.0
        self.eps_min = 0

        with tf.variable_scope('Actor'):
            l1 = tf.layers.dense(
                inputs=self.s,
                units=20,    # number of hidden units
                activation=tf.nn.relu,
                kernel_initializer=tf.random_normal_initializer(0., .1),    # weights
                bias_initializer=tf.constant_initializer(0.1),  # biases
                name='l1'
            )

            self.acts_prob = tf.layers.dense(
                inputs=l1,
                units=n_actions,    # output units
                activation=tf.contrib.sparsemax.sparsemax,   # get action probabilities
                kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
                bias_initializer=tf.constant_initializer(0.1),  # biases
                name='acts_prob'
            )

        with tf.variable_scope('exp_v'):
            self.error = self.td_error + ALPHA*0.5*tf.reduce_sum(
                tf.multiply(self.acts_prob, self.acts_prob), 1, keep_dims=True) + ALPHA*0.5 - ALPHA*tf.reduce_sum(
                tf.multiply(self.acts_prob, self.a), 1, keep_dims = True)
            # TD error + ALPHA(0.5*pi^2 + 0.5 - pi)
            self.loss = tf.reduce_sum(tf.square(self.error))

        with tf.variable_scope('train'):
            self.train_op = tf.train.AdamOptimizer(lr).minimize(self.loss)  

    def learn(self, s, a, td):
        feed_dict = {self.s: s, self.a: a, self.td_error: td}
        _, loss = self.sess.run([self.train_op, self.loss], feed_dict)
        return loss

    def choose_action(self, s):
        s = s[np.newaxis, :]
        probs = self.sess.run(self.acts_prob, {self.s: s})   # get probabilities for all actions
        probs = probs.ravel()
        probs = np.ones(probs.shape[0]) * self.eps / probs.shape[0] + probs * (1. - self.eps)
        probs = probs/np.sum(probs)
        return np.random.choice(np.arange(probs.shape[0]), p=probs)   # return a int
    
    def update_policy(self):
        self.eps = np.max((self.eps*self.eps_decay_rate, self.eps_min))

In [ ]:
class Critic(object):
    def __init__(self, sess, n_features, lr=0.01):
        self.sess = sess

        self.s = tf.placeholder(tf.float32, (None, n_features), "state")
        self.v_ = tf.placeholder(tf.float32, (None, 1), "v_next")
        self.r = tf.placeholder(tf.float32, (None, 1), 'r')

        with tf.variable_scope('Critic'):
            l1 = tf.layers.dense(
                inputs=self.s,
                units=20,  # number of hidden units
                activation=tf.nn.relu,  # None
                # have to be linear to make sure the convergence of actor.
                # But linear approximator seems hardly learns the correct Q.
                kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
                bias_initializer=tf.constant_initializer(0.1),  # biases
                name='l1'
            )

            self.v = tf.layers.dense(
                inputs=l1,
                units=1,  # output units
                activation=None,
                kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
                bias_initializer=tf.constant_initializer(0.1),  # biases
                name='V'
            )
            
        with tf.variable_scope('squared_TD_error'):
            self.td_error = self.r + GAMMA * self.v_ - self.v
            self.loss = tf.reduce_sum(tf.square(self.td_error))    # TD_error = (r+gamma*V_next) - V_eval
        with tf.variable_scope('train'):
            self.train_op = tf.train.AdamOptimizer(lr).minimize(self.loss)

    def learn(self, s, r, s_):
        v_ = self.sess.run(self.v, {self.s: s_})
        td_error, _ = self.sess.run([self.td_error, self.train_op],
                                          {self.s: s, self.v_: v_, self.r: r})
        return td_error
    
    def get_error(self, s, r, s_):
        v_ = self.sess.run(self.v, {self.s: s_})
        td_error = self.sess.run(self.td_error,
                                          {self.s: s, self.v_: v_, self.r: r})
        return td_error
        

In [ ]:
sess = tf.Session()

actor = Actor(sess, n_features=N_F, n_actions=N_A, lr=LR_A)
# critic = Critic(sess, n_features=N_F, lr=LR_C)
critic = Critic(sess, n_features=N_F, lr=LR_C)     # we need a good teacher, so the teacher should learn faster than the actor
MEMORY_SIZE = 10000
BATCH_SIZE = 1000
replay = ExpReplay(MEMORY_SIZE, N_F, N_A, BATCH_SIZE)
sess.run(tf.global_variables_initializer())

if OUTPUT_GRAPH:
    tf.summary.FileWriter("logs/", sess.graph)

for i_episode in range(MAX_EPISODE):
    s = env.reset()
    t = 0
    track_r = []
    while True:
        if RENDER: env.render()

        a = actor.choose_action(s)
        s_, r, done, info = env.step(a)

        if done: r = -20

        track_r.append(r)

        #td_error = critic.learn(s, r, s_)  # gradient = grad[r + gamma * V(s_) - V(s)]
        #actor.learn(s, a, td_error)     # true_gradient = grad[logPi(s,a) * td_error]
        replay.fifo(s, a, r, s_)
        s = s_
        t += 1

        if done or t >= MAX_EP_STEPS:
            ep_rs_sum = sum(track_r)

            if 'running_reward' not in globals():
                running_reward = ep_rs_sum
            else:
                running_reward = running_reward * 0.95 + ep_rs_sum * 0.05
            if running_reward > DISPLAY_REWARD_THRESHOLD: RENDER = False  # rendering
            print("episode:", i_episode, "  reward:", int(running_reward))
            break
    states, actions, rewards, next_states = replay.sampling()
    mx_actions = np.zeros((actions.shape[0], N_A))
    for i in range(0, states.shape[0]):
        mx_actions[i, int(actions[i, 0])] = 1
    td_errors = critic.get_error(states, rewards, next_states)
    actor.learn(states, mx_actions, td_errors)
    critic.learn(states, rewards, next_states)
    actor.update_policy()

('episode:', 0, '  reward:', -7)
('episode:', 1, '  reward:', -6)
('episode:', 2, '  reward:', -6)
('episode:', 3, '  reward:', -6)
('episode:', 4, '  reward:', -6)
('episode:', 5, '  reward:', -6)
('episode:', 6, '  reward:', -5)
('episode:', 7, '  reward:', -5)
('episode:', 8, '  reward:', -5)
('episode:', 9, '  reward:', -5)
('episode:', 10, '  reward:', -4)
('episode:', 11, '  reward:', -4)
('episode:', 12, '  reward:', -5)
('episode:', 13, '  reward:', -4)
('episode:', 14, '  reward:', -4)
('episode:', 15, '  reward:', -4)
('episode:', 16, '  reward:', -4)
('episode:', 17, '  reward:', -4)
('episode:', 18, '  reward:', -5)
('episode:', 19, '  reward:', -4)
('episode:', 20, '  reward:', -3)
('episode:', 21, '  reward:', -3)
('episode:', 22, '  reward:', -4)
('episode:', 23, '  reward:', -3)
('episode:', 24, '  reward:', -3)
('episode:', 25, '  reward:', -3)
('episode:', 26, '  reward:', -3)
('episode:', 27, '  reward:', -3)
('episode:', 28, '  reward:', -3)
('episode:', 29, '  rewa

('episode:', 241, '  reward:', 53)
('episode:', 242, '  reward:', 53)
('episode:', 243, '  reward:', 54)
('episode:', 244, '  reward:', 53)
